In [1]:
# This was my much newer, much improved, more mathematically sound implementation
# of a neural network. I've used proper numpy and matrices here.

# Forgive the bad naming: I was programming from a mathematical perspective
# with single-letter variable names.

In [2]:
import numpy as np
import pandas as pd
np.random.seed(1)

In [3]:
# Initialization of some of the neural network variables

n = np.array([784, 16, 16, 10]) # Layer sizes, incl input layer
L = len(n)-1 # number of layers, excluding input layer

train = pd.read_csv("train.csv")
X = train.iloc[:, 1:].to_numpy().T/255 # inputs
Y = train["label"].to_numpy() # expected outputs
one_hot_Y = np.zeros((Y.size, 10))
one_hot_Y[np.arange(Y.size), Y] = 1
Y = one_hot_Y.T

N = X.shape[1] # Number of training examples

A = [np.zeros((n[i], N)) for i in range(L+1)] # activation values at each layer (incl. input layer)
A[0] = X # first layer activations = input values

# We add some None's to be able to start indexing at 1
Z = [None] + [np.zeros((n[i], N)) for i in range(1, L+1)] # Start z values (intermediate weighted-sum + bias) at 0
W = [None] # Weights
B = [None] # Biases
for i in range(L):
    # Initialize weights and biases between -0.5 and 0.5
    W.append(np.random.rand(n[i+1], n[i]) - 0.5)
    B.append(np.random.rand(n[i+1], 1) - 0.5)

In [4]:
# Declare activation functions

def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_deriv(x):
    return sigmoid(x)*(1-sigmoid(x))

def softmax(x):
    epsilon = 10**-7
    return np.exp(x)/(sum(np.exp(x))+epsilon)

def softmax_deriv(x):
    return softmax(x)*(1-softmax(x))

def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return x > 0

class ActivationFunction:
    def __init__(self, function, derivative):
        self.activation = function
        self.derivative = derivative

# Activation functions at each layer
F = [
    None, # None so we can index starting at 1
    ActivationFunction(relu, relu_deriv),
    ActivationFunction(relu, relu_deriv),
    ActivationFunction(softmax, softmax_deriv),
]

In [5]:
def forward_prop(Z, A, W, B, F):
    for i in range(1, L+1):
        Z[i] = np.matmul(W[i], A[i-1]) + B[i]
        A[i] = F[i].activation(Z[i])
    return Z, A

def output(Z, A, W, B, F):
    Z, A = forward_prop(Z, A, W, B, F)
    return A[-1]

def error(A, Y):
    # We're using mean squared error as our error function
    square = lambda x: x**2
    return np.sum(square(A[-1]-Y)) # A[-1] used to get last layer

def error_deriv(A, Y): # derivative of error function
    return 2*(A[-1]-Y)

def backprop(Z, A, Y, W, B, F, N, learning_rate):
    Z, A = forward_prop(Z, A, W, B, F)
    dEdA = error_deriv(A, Y) # dE/dA = Derivative of error with respect to last layer activations
    for i in range(L, 0, -1): # go backwards through layer indices (backpropagation)
        dAdZ = F[i].derivative(Z[i])
        dEdZ = dEdA*dAdZ
        dEdA = np.matmul(W[i].T, dEdZ)
        B[i] -= learning_rate*1/N*np.sum(dEdZ, axis=1, keepdims=True) # dZdB = 1
        W[i] -= learning_rate*1/N*np.dot(dEdZ, A[i-1].T)

    return B, W

In [6]:
num_backprops = 1000
for i in range(1000):
    if i%100 == 0: 
        print(f"{i}/{num_backprops} backpropagations done")
    B, W = backprop(Z, A, Y, W, B, F, N, 1)

0
100
200
300
400
500
600
700
800
900


In [7]:
out = np.argmax(output(Z, A, W, B, F).T, axis=1)
print(out) # a row vector of all the MNIST digit predictions

[1 0 1 ... 7 6 9]


In [8]:
# Get the accuracy
print(np.sum(np.argmax(output(Z, A, W, B, F).T, axis=1) == train["label"])/X.shape[1])

0.9275714285714286
